In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.visualization import SqrtStretch, ImageNormalize, LinearStretch, quantity_support
import numpy as np
import matplotlib.pyplot as plt
from RMtools_3D.do_RMsynth_3D import run_rmsynth
from RMtools_3D.do_RMclean_3D import do_rmclean_hogbom
import multiprocessing as mp
_ = quantity_support()

Load in the data using standard Astropy FITS

In [ ]:
# Load the data
data = {}
for stokes in "IQU":
    data[stokes] = {}
    with fits.open(f"POSSUM_M83/cutout-M83_{stokes}.fits") as hdul:
        data[stokes]["data"] = hdul[0].data.squeeze()
        data[stokes]["header"] = hdul[0].header
        data[stokes]["wcs"] = WCS(data[stokes]["header"])


Visualise the data

In [ ]:
# Plot the first channel of the data
fig, ax = plt.subplots(1, 4, figsize=(15, 5), subplot_kw=dict(projection=data["I"]["wcs"].celestial), sharex=True, sharey=True)

for i, stokes in enumerate("IQU"):
    if stokes == "I":
        norm = ImageNormalize(data[stokes]["data"][0], stretch=SqrtStretch())
        cmap = "magma"
    else:
        norm = ImageNormalize(data[stokes]["data"][0], stretch=LinearStretch())
        cmap = "RdBu_r"
    ax[i].imshow(data[stokes]["data"][0], origin="lower", cmap=cmap, norm=norm)
    ax[i].set_title(stokes)
    ax[i].set_xlabel("RA")
    ax[i].set_ylabel("Dec")
    ax[i].colorbar = fig.colorbar(ax[i].images[0], ax=ax[i], orientation="horizontal")

pol_int = np.hypot(data["Q"]["data"][0], data["U"]["data"][0])
norm = ImageNormalize(pol_int, stretch=LinearStretch())
cmap = "magma"
ax[-1].imshow(
    pol_int, origin="lower", cmap=cmap, norm=norm
)
ax[-1].set_title("Polarized intensity")
ax[-1].set_xlabel("RA")
ax[-1].set_ylabel("Dec")
ax[-1].colorbar = fig.colorbar(ax[-1].images[0], ax=ax[-1], orientation="horizontal")
fig.tight_layout()

In [ ]:
stokes_I_mean.shape

In [ ]:
# Compute averages - NOT correct for polarisation...
stokes_I_mean = np.nanmean(data["I"]["data"], axis=0)
stokes_Q_mean = np.nanmean(data["Q"]["data"], axis=0)
stokes_U_mean = np.nanmean(data["U"]["data"], axis=0)
pol_int_mean = np.hypot(stokes_Q_mean, stokes_U_mean)

# Plot the mean data
fig, ax = plt.subplots(1, 4, figsize=(15, 5), subplot_kw=dict(projection=data["I"]["wcs"].celestial), sharex=True, sharey=True)
ax[0].imshow(stokes_I_mean, origin="lower", cmap="magma", norm=ImageNormalize(stokes_I_mean, stretch=SqrtStretch()))
ax[0].set_title("Mean Stokes I")
ax[0].set_xlabel("RA")
ax[0].set_ylabel("Dec")
ax[0].colorbar = fig.colorbar(ax[0].images[0], ax=ax[0], orientation="horizontal")
ax[1].imshow(stokes_Q_mean, origin="lower", cmap="RdBu_r", norm=ImageNormalize(stokes_Q_mean, stretch=LinearStretch()))
ax[1].set_title("Mean Stokes Q")
ax[1].set_xlabel("RA")
ax[1].set_ylabel("Dec")
ax[1].colorbar = fig.colorbar(ax[1].images[0], ax=ax[1], orientation="horizontal")
ax[2].imshow(stokes_U_mean, origin="lower", cmap="RdBu_r", norm=ImageNormalize(stokes_U_mean, stretch=LinearStretch()))
ax[2].set_title("Mean Stokes U")
ax[2].set_xlabel("RA")
ax[2].set_ylabel("Dec")
ax[2].colorbar = fig.colorbar(ax[2].images[0], ax=ax[2], orientation="horizontal")
ax[3].imshow(pol_int_mean, origin="lower", cmap="magma", norm=ImageNormalize(pol_int_mean, stretch=LinearStretch()))
ax[3].set_title("Mean Polarized intensity")
ax[3].set_xlabel("RA")
ax[3].set_ylabel("Dec")
ax[3].colorbar = fig.colorbar(ax[3].images[0], ax=ax[3], orientation="horizontal")
fig.tight_layout()


# Run RM-Synth

In [ ]:
# Check the arguments of run_rmsynth
run_rmsynth?

In [ ]:
# Run RM-synthesis
FDFcube, phiArr_radm2, RMSFcube, phi2Arr_radm2, fwhmRMSFCube,fitStatArr, lam0Sq_m2, lambdaSqArr_m2 = run_rmsynth(
    dataQ = data["Q"]["data"],
    dataU = data["U"]["data"],
    freqArr_Hz=data["I"]["wcs"].spectral.pixel_to_world(np.arange(data["I"]["data"].shape[0])).to(u.Hz).value,
    phiMax_radm2=100,
    verbose=True,
)
rmsf_unit = u.def_unit("RMSF")
FDFcube *= u.Unit(data["I"]["header"]["BUNIT"]) / rmsf_unit
fwhmRMSFCube *= u.rad / u.m**2
phiArr_radm2 *= u.rad / u.m**2
delta_phi = (phiArr_radm2[1] - phiArr_radm2[0]) / u.pix
assert np.allclose(np.diff(phiArr_radm2)/u.pix, delta_phi)

# Could also run the following on the command line
```bash
rmtools_freqfile POSSUM_M83/cutout-M83_I.fits POSSUM_M83/freqs.txt
rmsynth3d POSSUM_M83/cutout-M83_Q.fits POSSUM_M83/cutout-M83_U.fits POSSUM_M83/freqs.txt -l 100 -v
```

In [ ]:
# Compute the Faraday moments with correct units
pix_per_rmsf = (fwhmRMSFCube / delta_phi) / rmsf_unit
FDF_per_pixel = (FDFcube / pix_per_rmsf)
moment_0 = np.nansum(np.abs(FDF_per_pixel), axis=0) * 1* u.pixel
moment_1 = np.nansum(np.abs(FDF_per_pixel) * phiArr_radm2[:,None,None], axis=0) * 1 * u.pixel / moment_0
moment_2 = np.nansum(np.abs(FDF_per_pixel) * (phiArr_radm2[:,None,None] - moment_1)**2, axis=0) * 1 * u.pixel / moment_0

In [ ]:
moment_0.unit

In [ ]:
moment_1.unit

In [ ]:
moment_2.unit

Visualise the moments

In [ ]:
# Plot the moments
fig, ax = plt.subplots(1, 3, figsize=(15, 5), subplot_kw=dict(projection=data["I"]["wcs"].celestial), sharex=True, sharey=True)
ax[0].imshow(moment_0.value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[0].set_title("Moment 0")
ax[0].set_xlabel("RA")
ax[0].set_ylabel("Dec")
ax[0].colorbar = fig.colorbar(ax[0].images[0], ax=ax[0], orientation="horizontal")

ax[1].imshow(moment_1.to(u.rad/u.m**2).value, origin="lower", cmap="RdBu_r", norm=ImageNormalize(stretch=LinearStretch()))
ax[1].set_title("Moment 1")
ax[1].set_xlabel("RA")
ax[1].set_ylabel("Dec")
ax[1].colorbar = fig.colorbar(ax[1].images[0], ax=ax[1], orientation="horizontal")

ax[2].imshow(moment_2.to(u.rad**2/u.m**4).value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[2].set_title("Moment 2")
ax[2].set_xlabel("RA")
ax[2].set_ylabel("Dec")
ax[2].colorbar = fig.colorbar(ax[2].images[0], ax=ax[2], orientation="horizontal")


# Further excerises:
- Mask the the moment 1 and 2 maps to only show the regions where the moment 0 is above a noise threshold
- Compute the fractional polarisation
- Run the equivalent of the above on the commandline and read the input products into python
- Run the equivalent of the above on the commandline and read the input products into CARTA

# Now onto RM-CLEAN

In [ ]:
# Check the arguments of run_rmclean
do_rmclean_hogbom?

In [ ]:
cleanFDF, ccArr, iterCountArr, residFDF = do_rmclean_hogbom(
    dirtyFDF=FDFcube.value,
    phiArr_radm2=phiArr_radm2.value,
    RMSFArr=RMSFcube,
    phi2Arr_radm2=phi2Arr_radm2,
    fwhmRMSFArr=fwhmRMSFCube.value,
    cutoff=0.0003,
    verbose=True,
)

# Could also run the following on the command line
```bash
rmclean3d POSSUM_M83/FDF_dirty.fits POSSUM_M83/RMSF.fits -c 0.0003 -v
```

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw=dict(projection=data["I"]["wcs"].celestial), sharex=True, sharey=True)
im = ax.imshow(iterCountArr, origin="lower", norm=ImageNormalize(stretch=LinearStretch(), vmin=0, vmax=iterCountArr.max()))
ax.set_title("Number of iterations")
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
ax.colorbar = fig.colorbar(im, ax=ax, orientation="horizontal")

In [ ]:
# Compute moments of outputs
cleanFDF *= u.Unit(data["I"]["header"]["BUNIT"]) / rmsf_unit
ccArr *= u.Unit(data["I"]["header"]["BUNIT"]) / rmsf_unit
FDF_per_pixel = (cleanFDF / pix_per_rmsf)
ccArr_per_pixel = (ccArr / pix_per_rmsf)

moment_0_clean = np.nansum(np.abs(FDF_per_pixel), axis=0) * 1* u.pixel
moment_1_clean = np.nansum(np.abs(FDF_per_pixel) * phiArr_radm2[:,None,None], axis=0) * 1 * u.pixel / moment_0_clean
moment_2_clean = np.nansum(np.abs(FDF_per_pixel) * (phiArr_radm2[:,None,None] - moment_1_clean)**2, axis=0) * 1 * u.pixel / moment_0_clean

moment_0_cc = np.nansum(np.abs(ccArr_per_pixel), axis=0) * 1* u.pixel
moment_1_cc = np.nansum(np.abs(ccArr_per_pixel) * phiArr_radm2[:,None,None], axis=0) * 1 * u.pixel / moment_0_cc
moment_2_cc = np.nansum(np.abs(ccArr_per_pixel) * (phiArr_radm2[:,None,None] - moment_1_cc)**2, axis=0) * 1 * u.pixel / moment_0_cc


In [ ]:
# Plot the moments
fig, ax = plt.subplots(2, 3, figsize=(15, 10), subplot_kw=dict(projection=data["I"]["wcs"].celestial), sharex=True, sharey=True)
ax[0,0].imshow(moment_0_clean.value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[0,0].set_title("Moment 0 (clean)")
ax[0,0].set_xlabel("RA")
ax[0,0].set_ylabel("Dec")
ax[0,0].colorbar = fig.colorbar(ax[0,0].images[0], ax=ax[0,0], orientation="horizontal")

ax[0,1].imshow(moment_1_clean.to(u.rad/u.m**2).value, origin="lower", cmap="RdBu_r", norm=ImageNormalize(stretch=LinearStretch()))
ax[0,1].set_title("Moment 1 (clean)")
ax[0,1].set_xlabel("RA")
ax[0,1].set_ylabel("Dec")
ax[0,1].colorbar = fig.colorbar(ax[0,1].images[0], ax=ax[0,1], orientation="horizontal")

ax[0,2].imshow(moment_2_clean.to(u.rad**2/u.m**4).value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[0,2].set_title("Moment 2 (clean)")
ax[0,2].set_xlabel("RA")
ax[0,2].set_ylabel("Dec")
ax[0,2].colorbar = fig.colorbar(ax[0,2].images[0], ax=ax[0,2], orientation="horizontal")

ax[1,0].imshow(moment_0_cc.value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[1,0].set_title("Moment 0 (CC)")
ax[1,0].set_xlabel("RA")
ax[1,0].set_ylabel("Dec")
ax[1,0].colorbar = fig.colorbar(ax[1,0].images[0], ax=ax[1,0], orientation="horizontal")

ax[1,1].imshow(moment_1_cc.to(u.rad/u.m**2).value, origin="lower", cmap="RdBu_r", norm=ImageNormalize(stretch=LinearStretch()))
ax[1,1].set_title("Moment 1 (CC)")
ax[1,1].set_xlabel("RA")
ax[1,1].set_ylabel("Dec")
ax[1,1].colorbar = fig.colorbar(ax[1,1].images[0], ax=ax[1,1], orientation="horizontal")

ax[1,2].imshow(moment_2_cc.to(u.rad**2/u.m**4).value, origin="lower", cmap="magma", norm=ImageNormalize(stretch=SqrtStretch()))
ax[1,2].set_title("Moment 2 (CC)")
ax[1,2].set_xlabel("RA")
ax[1,2].set_ylabel("Dec")
ax[1,2].colorbar = fig.colorbar(ax[1,2].images[0], ax=ax[1,2], orientation="horizontal")



fig.tight_layout()



# Further excerises:
- Compute the appropriate cutoff level for RM-clean
- Compare the CLEAN and Dirty spectra
- Run the equivalent of the above on the commandline and read the input products into python
- Run the equivalent of the above on the commandline and read the input products into CARTA